In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_excel('pivot_table_test2.xlsx')
df.columns = ['id', '単語×品詞', '単語']
df

,id,単語×品詞,単語
0,1,弱み&標準_名詞-一般,弱み
1,1,弱み_名詞-一般,弱み
2,1,強み_名詞-一般,強み
3,1,店舗_名詞-一般,店舗
4,1,強み_名詞-一般,強み
5,2,弱み_名詞-一般,弱み
6,2,真珠_名詞-一般,真珠
7,2,店舗_名詞-一般,店舗
8,2,手薄_名詞-一般,手薄
9,3,天候_名詞-一般,天候


## ID毎に単語をまとめる
単語と単語の間に半角スペースを挿入する

In [3]:
summary_df = df.groupby('id')['単語×品詞'].apply(lambda x: ' '.join(map(str, x)))\
.reset_index().rename(columns={'単語×品詞':'split_words'})

summary_df

,id,split_words
0,1,弱み&標準_名詞-一般 弱み_名詞-一般 強み_名詞-一般 店舗_名詞-一般 強み_名詞-一般
1,2,弱み_名詞-一般 真珠_名詞-一般 店舗_名詞-一般 手薄_名詞-一般
2,3,天候_名詞-一般 天候_名詞-一般 真珠_名詞-一般 弱み。_名詞-一般


# TFIDFの算出(TfidfVectorizerを使用)

In [4]:
# 単語と単語を半角スペースで分割(=token_pattern=r'\b[^\d\ ]+\b')
# デフォルトだと「&」や「_」で分割されてしまうので上記表現に変更する
# 参考URL：https://datascience.stackexchange.com/questions/29352/sklearn-countvectorizer-token-pattern-skip-token-if-pattern-match
vectorizer= TfidfVectorizer(token_pattern=r'\b[^\d\ ]+\b')

# TfidfVectorizerを利用して単語と列番号の対応付けを実行 Document-Term Matrixを獲得できる
tfidf_feature_vectors = vectorizer.fit_transform(summary_df['split_words'])

# どの単語を学習しているのかをvectorizer.get_feature_names()で確認できる
# 後にデータフレームを作成する際にカラムとして使用する準備
tfidf_vocabulary = vectorizer.get_feature_names()

tfidf_vocabulary

['天候_名詞-一般',
 '店舗_名詞-一般',
 '弱み&標準_名詞-一般',
 '弱み_名詞-一般',
 '弱み。_名詞-一般',
 '強み_名詞-一般',
 '手薄_名詞-一般',
 '真珠_名詞-一般']

In [5]:
df['単語×品詞'].nunique()

8

In [6]:
# ['単語×品詞']カラム重複なし単語
list(set(df['単語×品詞'].tolist()))

['手薄_名詞-一般',
 '天候_名詞-一般',
 '店舗_名詞-一般',
 '強み_名詞-一般',
 '真珠_名詞-一般',
 '弱み_名詞-一般',
 '弱み。_名詞-一般',
 '弱み&標準_名詞-一般']

# BoW作成

In [7]:
tfidf_df = pd.DataFrame(tfidf_feature_vectors.toarray(), index=summary_df['id'],  columns=tfidf_vocabulary)

# 確認
tfidf_df

,天候_名詞-一般,店舗_名詞-一般,弱み&標準_名詞-一般,弱み_名詞-一般,弱み。_名詞-一般,強み_名詞-一般,手薄_名詞-一般,真珠_名詞-一般
id,,,,,,,,
1,0.000000,0.306504,0.403016,0.306504,0.000000,0.806032,0.000000,0.000000
2,0.000000,0.459854,0.000000,0.459854,0.000000,0.000000,0.604652,0.459854
3,0.846789,0.000000,0.000000,0.000000,0.423394,0.000000,0.000000,0.322002
